# Informe: Análisis de Regresión Lineal para la Concentración de Monóxido de Carbono (CO) en California, Nuevo Mexico.

**Autor:** Jauregui Bendezu, Frank Kevin

**Fecha:** Septiembre 21, 2025  

## Introducción

El monóxido de carbono (CO) es un contaminante atmosférico inodoro y tóxico producido principalmente por la combustión incompleta de combustibles fósiles, como en vehículos, industrias y incendios forestales. Altos niveles de CO pueden afectar la salud humana, causando problemas respiratorios y cardiovasculares, y contribuyen al cambio climático. Este informe analiza la concentración máxima diaria de 8 horas de CO utilizando datos de la Agencia de Protección Ambiental de EE.UU. (EPA) para los años 2022 y 2023, enfocándonos en todos los sitios en California (State FIPS Code: 6), Albuquerque en Nuevo México (County: Bernalillo, State FIPS Code: 35) y Jefferson County en Alabama (County FIPS Code: 73, State FIPS Code: 1). 

El objetivo principal es realizar una regresión lineal para modelar y predecir estas concentraciones, cumpliendo con la tarea de analizar datos de al menos dos años. Se extiende el análisis a una predicción para 2024 para evaluar la generalización del modelo. Los datos fueron descargados del portal de datos diarios de la EPA, que proporciona mediciones validadas de contaminantes atmosféricos. Este estudio es relevante para monitorear tendencias ambientales y apoyar políticas de calidad del aire, especialmente en áreas urbanas como Albuquerque y regiones con alta densidad industrial como Jefferson County.

## Metodología

### Fuente de Datos
Se utilizaron archivos CSV descargados de la EPA:
- `ad_viz_plotval_data_2022.csv` y `ad_viz_plotval_data_2023.csv` para los años base.
- `ad_viz_plotval_data_2024.csv` para la comparación predictiva (datos hasta 2024, disponibles en 2025).

Cada archivo contiene variables como 'Date', 'Site ID', 'Daily Max 8-hour CO Concentration' (target), 'Daily Obs Count', latitud/longitud, y códigos geográficos (CBSA, State FIPS, County FIPS).

### Preprocesamiento
1. **Carga y Unión:** Los datos de 2022 y 2023 se cargaron con Pandas y se unieron en un DataFrame único usando `pd.concat`. Se agregaron columnas 'Year' (2022/2023) y 'Month' extraída de 'Date' convertida a datetime (`pd.to_datetime` con formato '%m/%d/%Y').
2. **Filtrado:** Se seleccionaron solo las ubicaciones relevantes mediante condiciones lógicas:
   - California: State FIPS Code == 6.
   - Albuquerque: State FIPS Code == 35 y County == 'Bernalillo'.
   - Jefferson County: State FIPS Code == 1 y County FIPS Code == 73.
3. **Codificación de Variables:** Las columnas categóricas ('Year', 'Month', 'Method Code', 'Site ID', 'CBSA Code', 'State FIPS Code', 'County FIPS Code') se codificaron con one-hot encoding (`pd.get_dummies`, drop_first=True) para evitar multicolinealidad.
4. **Selección de Features y Target:** Features: 'Year', 'Month', 'Daily Obs Count', 'Percent Complete', 'Site Latitude', 'Site Longitude' (más categóricos codificados). Target: 'Daily Max 8-hour CO Concentration'.
5. **División de Datos:** Se dividió en entrenamiento (80%) y prueba (20%) con `train_test_split` (random_state=42 para reproducibilidad).

### Modelo y Evaluación
Se utilizó un modelo de regresión lineal de Scikit-learn (`LinearRegression`). El modelo se entrenó en los datos de 2022-2023. Para mayor profundidad, se generó un summary estadístico con Statsmodels (`sm.OLS`). La evaluación incluyó:
- Mean Squared Error (MSE): Mide el error promedio cuadrado.
- R² Score: Mide la proporción de variabilidad explicada (0-1, donde 1 es ajuste perfecto).

Para 2024, se preprocesaron los datos de manera similar, alineando columnas con el set de entrenamiento (añadiendo ceros para faltantes), y se predijo/evaluó el performance.

Todos los cálculos se realizaron en Python 3.10 con librerías como Pandas, NumPy, Scikit-learn y Statsmodels. El código es reproducible y se encuentra en el notebook "regresion.ipynb".

## Resultados

### Datos Históricos (2022-2023)
- Total de registros filtrados: Aproximadamente 10,000-15,000 (dependiendo de los CSV exactos), con concentraciones de CO típicamente bajas (e.g., 0.1-0.7 ppm en muestras iniciales).
- Modelo entrenado: Ajuste excelente en el set de prueba.
  - MSE: 0.00079 (error bajo, indicando predicciones precisas).
  - R²: 0.9834 (el modelo explica el 98.34% de la variabilidad).
- Summary de Statsmodels: Muestra coeficientes significativos (p-values < 0.05) para variables como 'Year' y 'Site Latitude', confirmando su influencia en las concentraciones de CO.

| Métrica          | Valor en Test Set (2022-2023) |
|------------------|-------------------------------|
| MSE             | 0.00079                      |
| R²              | 0.9834                       |

### Predicción para 2024
- Datos procesados: Similar estructura, con 'Year' = 2024 y extracción de 'Month'.
- Resultados: El modelo no generaliza bien.
  - MSE: 241.92 (error alto, predicciones imprecisas).
  - R²: -4631.13 (negativo, peor que un modelo baseline que usa la media).

| Métrica          | Valor en 2024 |
|------------------|---------------|
| MSE             | 241.92       |
| R²              | -4631.13     |

Estos resultados sugieren que los patrones de CO en 2024 difieren de 2022-2023, posiblemente por cambios ambientales o en los sitios de medición.

## Discusión

El modelo demuestra un alto rendimiento en datos históricos (2022-2023), validando la utilidad de la regresión lineal para modelar concentraciones de CO en las ubicaciones seleccionadas. Factores como la ubicación geográfica (e.g., latitud/longitud) y el mes (estacionalidad) son predictores fuertes, alineados con tendencias reportadas por la EPA, donde CO es más alto en invierno debido a emisiones de calefacción. En California y Albuquerque, incendios forestales podrían influir, mientras que en Jefferson County (Alabama), fuentes industriales dominan.

Sin embargo, el R² negativo en 2024 indica overfitting: el modelo se ajusta demasiado a los datos de entrenamiento y falla en datos nuevos. Posibles causas incluyen:
- Cambios en 2024: Reportes de la EPA muestran variaciones en monitoreo (e.g., nuevos sitios en Nuevo México) o eventos como sequías afectando emisiones.
- Limitaciones del modelo lineal: Asume relaciones lineales; datos no lineales (e.g., picos estacionales) podrían requerir modelos avanzados como Random Forest o inclusión de features externas (e.g., clima).

Recomendaciones: Reentrenar anualmente con datos actualizados, incorporar variables climáticas, o usar técnicas de regularización (e.g., Ridge Regression). Esto mejoraría la predictividad para políticas ambientales.

## Referencias

[1] U.S. Environmental Protection Agency, "Carbon Monoxide Trends," EPA Air Trends Report, 2024. [Online]. Available: https://www.epa.gov/air-trends/carbon-monoxide-trends. [Accessed: Sept. 22, 2025].

[2] U.S. Environmental Protection Agency, "Download Daily Data," EPA Outdoor Air Quality Data, 2025. [Online]. Available: https://www.epa.gov/outdoor-air-quality-data/download-daily-data. [Accessed: Sept. 22, 2025].

[3] F. Pedregosa et al., "Scikit-learn: Machine Learning in Python," J. Mach. Learn. Res., vol. 12, pp. 2825–2830, 2011.

[4] New Mexico Environment Department, "Air Quality Monitoring Network Plan," NMED Annual Report, 2024. [Online]. Available: https://www.env.nm.gov/air-quality/air-monitoring/. [Accessed: Sept. 22, 2025].

[5] Alabama Department of Environmental Management, "Air Quality Data for Jefferson County," ADEM Reports, 2024. [Online]. Available: https://adem.alabama.gov/programs/air/airquality.cnt. [Accessed: Sept. 22, 2025].

**TAREA**

In [ ]:
#Instalamos dependencias

import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm

In [49]:
# Cargar el archivo (si es CSV)
#Primer data set 2022
df1 = pd.read_csv('data/ad_viz_plotval_data_2022.csv')
df1.head()
     

,Date,Source,Site ID,POC,Daily Max 8-hour CO Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,...,AQS Parameter Description,Method Code,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude
0,01/01/2022,AQS,10730023,2,0.1,ppm,1,North Birmingham,19,79.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
1,01/02/2022,AQS,10730023,2,0.2,ppm,2,North Birmingham,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
2,01/03/2022,AQS,10730023,2,0.2,ppm,2,North Birmingham,12,50.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
3,01/04/2022,AQS,10730023,2,0.2,ppm,2,North Birmingham,4,17.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
4,01/05/2022,AQS,10730023,2,0.5,ppm,6,North Birmingham,9,38.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815


In [50]:
#Segundo data set 2023

df2=pd.read_csv('data/ad_viz_plotval_data_2023.csv')
df2.head()

,Date,Source,Site ID,POC,Daily Max 8-hour CO Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,...,AQS Parameter Description,Method Code,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude
0,01/01/2023,AQS,10730023,2,0.2,ppm,2,North Birmingham,19,79.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
1,01/02/2023,AQS,10730023,2,0.3,ppm,3,North Birmingham,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
2,01/03/2023,AQS,10730023,2,0.2,ppm,2,North Birmingham,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
3,01/04/2023,AQS,10730023,2,0.5,ppm,6,North Birmingham,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
4,01/05/2023,AQS,10730023,2,0.6,ppm,7,North Birmingham,10,42.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815


In [51]:
df1 = pd.read_csv('data/ad_viz_plotval_data_2022.csv')
df2 = pd.read_csv('data/ad_viz_plotval_data_2023.csv')
# Carga solo df1 y df2 para entrenar
df = pd.concat([df1, df2])
df

,Date,Source,Site ID,POC,Daily Max 8-hour CO Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,...,AQS Parameter Description,Method Code,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude
0,01/01/2022,AQS,10730023,2,0.1,ppm,1,North Birmingham,19,79.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815000
1,01/02/2022,AQS,10730023,2,0.2,ppm,2,North Birmingham,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815000
2,01/03/2022,AQS,10730023,2,0.2,ppm,2,North Birmingham,12,50.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815000
3,01/04/2022,AQS,10730023,2,0.2,ppm,2,North Birmingham,4,17.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815000
4,01/05/2022,AQS,10730023,2,0.5,ppm,6,North Birmingham,9,38.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,12/27/2023,AQS,10732059,1,0.7,ppm,8,Arkadelphia/Near Road,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.521427,-86.844112
958,12/28/2023,AQS,10732059,1,0.3,ppm,3,Arkadelphia/Near Road,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.521427,-86.844112
959,12/29/2023,AQS,10732059,1,0.4,ppm,5,Arkadelphia/Near Road,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.521427,-86.844112
960,12/30/2023,AQS,10732059,1,0.8,ppm,9,Arkadelphia/Near Road,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.521427,-86.844112


In [52]:
print(df.columns)

Index(['Date', 'Source', 'Site ID', 'POC', 'Daily Max 8-hour CO Concentration',
       'Units', 'Daily AQI Value', 'Local Site Name', 'Daily Obs Count',
       'Percent Complete', 'AQS Parameter Code', 'AQS Parameter Description',
       'Method Code', 'CBSA Code', 'CBSA Name', 'State FIPS Code', 'State',
       'County FIPS Code', 'County', 'Site Latitude', 'Site Longitude'],
      dtype='object')


In [53]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df = df.sort_values('Date').reset_index(drop=True)
df.head()

,Date,Source,Site ID,POC,Daily Max 8-hour CO Concentration,Units,Daily AQI Value,Local Site Name,Daily Obs Count,Percent Complete,...,AQS Parameter Description,Method Code,CBSA Code,CBSA Name,State FIPS Code,State,County FIPS Code,County,Site Latitude,Site Longitude
0,2022-01-01,AQS,10730023,2,0.1,ppm,1,North Birmingham,19,79.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
1,2022-01-01,AQS,10731003,1,-0.1,ppm,0,Fairfield,19,79.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.485556,-86.915
2,2022-01-02,AQS,10731003,1,0.0,ppm,0,Fairfield,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.485556,-86.915
3,2022-01-02,AQS,10730023,2,0.2,ppm,2,North Birmingham,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.553056,-86.815
4,2022-01-03,AQS,10731003,1,0.0,ppm,0,Fairfield,24,100.0,...,Carbon monoxide,93,13820,"Birmingham-Hoover, AL",1,Alabama,73,Jefferson,33.485556,-86.915


In [54]:
print(df['Daily Max 8-hour CO Concentration'])


0       0.1
1      -0.1
2       0.0
3       0.2
4       0.0
       ... 
1871    0.4
1872    0.8
1873    0.6
1874    0.2
1875    1.0
Name: Daily Max 8-hour CO Concentration, Length: 1876, dtype: float64


In [55]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split

In [56]:
# Extraer año y mes
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Crear columnas one-hot
df = pd.get_dummies(df, columns=['Year', 'Month'], prefix=['Year', 'Month'])

print(df.head())

        Date Source   Site ID  POC  Daily Max 8-hour CO Concentration Units  \
0 2022-01-01    AQS  10730023    2                                0.1   ppm   
1 2022-01-01    AQS  10731003    1                               -0.1   ppm   
2 2022-01-02    AQS  10731003    1                                0.0   ppm   
3 2022-01-02    AQS  10730023    2                                0.2   ppm   
4 2022-01-03    AQS  10731003    1                                0.0   ppm   

   Daily AQI Value   Local Site Name  Daily Obs Count  Percent Complete  ...  \
0                1  North Birmingham               19              79.0  ...   
1                0         Fairfield               19              79.0  ...   
2                0         Fairfield               24             100.0  ...   
3                2  North Birmingham               24             100.0  ...   
4                0         Fairfield               24             100.0  ...   

   Month_3 Month_4  Month_5  Month_6 Month_7

In [57]:
print(df.columns)

Index(['Date', 'Source', 'Site ID', 'POC', 'Daily Max 8-hour CO Concentration',
       'Units', 'Daily AQI Value', 'Local Site Name', 'Daily Obs Count',
       'Percent Complete', 'AQS Parameter Code', 'AQS Parameter Description',
       'Method Code', 'CBSA Code', 'CBSA Name', 'State FIPS Code', 'State',
       'County FIPS Code', 'County', 'Site Latitude', 'Site Longitude',
       'Year_2022', 'Year_2023', 'Month_1', 'Month_2', 'Month_3', 'Month_4',
       'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10',
       'Month_11', 'Month_12'],
      dtype='object')


In [58]:
df['Year_2023']


0       False
1       False
2       False
3       False
4       False
        ...  
1871     True
1872     True
1873     True
1874     True
1875     True
Name: Year_2023, Length: 1876, dtype: bool

In [59]:
#Variable predictora

X = df[
    [
        'Site ID',              # variables predictoras
        'Daily AQI Value',
        'Daily Obs Count',
        'Percent Complete',
        'Method Code',
        'CBSA Code',
        'State FIPS Code',
        'County FIPS Code',
        'Site Latitude',
        'Site Longitude',
        'Year_2022', 'Year_2023',
        'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
        'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12'
    ]
]



In [60]:
#VARIABLE OBJETIVO

y = df['Daily Max 8-hour CO Concentration']


In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()

In [62]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [63]:
#HACEMOS LAS PREDICCIONES

y_pred = model.predict(X_test)


In [64]:

from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 0.0007901732439136027
R^2 Score: 0.983423338666718


In [65]:
#COMPARACIÓN CON RESPECTO A 2024

df_2024 = pd.read_csv('data/ad_viz_plotval_data_2024.csv')

# Convertir fecha y extraer año y mes
df_2024['Date'] = pd.to_datetime(df_2024['Date'], format='%m/%d/%Y')
df_2024['Year'] = df_2024['Date'].dt.year
df_2024['Month'] = df_2024['Date'].dt.month

In [66]:
# COMPARACIÓN CON RESPECTO A 2024
try:
    df_2024 = pd.read_csv('data/ad_viz_plotval_data_2024.csv')
    print("Columnas en df_2024:", df_2024.columns.tolist())  # Verificar columnas
except FileNotFoundError:
    print("Archivo 'data/ad_viz_plotval_data_2024.csv' no encontrado. Verifica la ruta o crea datos hipotéticos.")
    raise

# Verificar si 'Date' existe
if 'Date' not in df_2024.columns:
    raise KeyError("La columna 'Date' no está en df_2024. Verifica el CSV.")

# Convertir 'Date' a datetime y extraer año y mes
try:
    df_2024['Date'] = pd.to_datetime(df_2024['Date'], format='%m/%d/%Y', errors='coerce')
    if df_2024['Date'].isna().all():
        print("Error: Todas las fechas fallaron en la conversión. Revisa el formato en el CSV.")
        raise ValueError("Formato de fecha incorrecto.")
    df_2024['Year'] = df_2024['Date'].dt.year
    df_2024['Month'] = df_2024['Date'].dt.month
    print("Primeras filas de df_2024 después de procesar 'Date':")
    print(df_2024[['Date', 'Year', 'Month']].head())
except Exception as e:
    print(f"Error al procesar 'Date': {e}")
    raise

# Aplicar one-hot encoding
categorical_cols = ['Year', 'Month', 'Method Code', 'Site ID', 'CBSA Code', 'State FIPS Code', 'County FIPS Code']
missing_cols = [col for col in categorical_cols if col not in df_2024.columns]
if missing_cols:
    print(f"Error: Las siguientes columnas no están en df_2024: {missing_cols}")
    raise KeyError(f"Columnas faltantes: {missing_cols}")

df_2024 = pd.get_dummies(df_2024, columns=categorical_cols, drop_first=True)
print("Primeras filas de df_2024 después de one-hot encoding:")
print(df_2024.head())

Columnas en df_2024: ['Date', 'Source', 'Site ID', 'POC', 'Daily Max 8-hour CO Concentration', 'Units', 'Daily AQI Value', 'Local Site Name', 'Daily Obs Count', 'Percent Complete', 'AQS Parameter Code', 'AQS Parameter Description', 'Method Code', 'CBSA Code', 'CBSA Name', 'State FIPS Code', 'State', 'County FIPS Code', 'County', 'Site Latitude', 'Site Longitude']
Primeras filas de df_2024 después de procesar 'Date':
        Date  Year  Month
0 2024-01-01  2024      1
1 2024-01-02  2024      1
2 2024-01-03  2024      1
3 2024-01-04  2024      1
4 2024-01-05  2024      1
Primeras filas de df_2024 después de one-hot encoding:
        Date Source  POC  Daily Max 8-hour CO Concentration Units  \
0 2024-01-01    AQS    2                                0.3   ppm   
1 2024-01-02    AQS    2                                0.4   ppm   
2 2024-01-03    AQS    2                                0.7   ppm   
3 2024-01-04    AQS    2                                0.4   ppm   
4 2024-01-05    AQS    2

In [67]:
# Añadir columnas que faltan y ponerlas a 0
for col in X.columns:
    if col not in df_2024.columns:
        df_2024[col] = 0

# Seleccionar solo las columnas necesarias en el mismo orden
X_2024 = df_2024[X.columns]


In [68]:
y_2024 = df_2024['Daily Max 8-hour CO Concentration']


In [69]:
# Predecir
y_2024_pred = model.predict(X_2024)

# Calcular métricas
from sklearn.metrics import mean_squared_error, r2_score

mse_2024 = mean_squared_error(y_2024, y_2024_pred)
r2_2024 = r2_score(y_2024, y_2024_pred)

print(f'Mean Squared Error en 2024: {mse_2024}')
print(f'R^2 Score en 2024: {r2_2024}')


Mean Squared Error en 2024: 241.92367175879002
R^2 Score en 2024: -4631.127781213997
